### This code scrapes product data from eBay and saves it to a CSV file.

In [1]:
# Import necessary modules and packages
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Create a new Chrome browser instance
driver = webdriver.Chrome()

# Navigate to the eBay website
driver.get('https://www.ebay.com/')

# Create an empty list to store product links
links = []

In [3]:
# Select the 'Computers/Tablets & Networking' category from the dropdown menu
dropdown = Select(driver.find_element(By.ID, 'gh-cat'))
dropdown.select_by_visible_text('Computers/Tablets & Networking')

In [4]:
# Search for 'lenovo small screen' in the search box
search_box = driver.find_element(By.XPATH, '//*[@id="gh-ac"]')
search_box.send_keys('lenovo small screen')
search_box.send_keys(Keys.RETURN)

In [5]:
# Calculate the number of products and pages
num_of_products = driver.find_element(By.XPATH, '//*[@id="mainContent"]/div[1]/div/div[2]/div[1]/div[1]/h1/span[1]').text
num_of_products = num_of_products.replace(',', '')  # remove comma
num_of_pages = int(int(num_of_products) / 60)
print('Number of Products:',num_of_products)
print('Number of Pages:',num_of_pages)

Number of Products: 159
Number of Pages: 2


In [6]:
for x in range(1, num_of_pages +1):
    # Find all the <a> tags that contain product links
    product_links = driver.find_elements(By.XPATH, '//a[@class="s-item__link"]')

    # Loop through the <a> tags and extract the product links
    for link in product_links:
        # Extract the product link
        href = link.get_attribute('href')
        links.append(href)

In [7]:
# Close the browser instance
driver.quit()

In [8]:
# Create empty lists to store product data
titles = []
prices = []
conditions = []
quantities = []
shippings = []
import_charges = []
deliveries = []
return_policies = []

In [9]:
# Loop through the links and scrape data from each page
for link in links:
    # Send a GET request to the URL
    response = requests.get(link)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the data you need from the page
    product_title = soup.find('h1', class_='x-item-title__mainTitle')
    if product_title:
        product_title_text = product_title.text.strip()
    else:
        product_title_text = 'N/A'

    product_price = soup.find('span', itemprop='price')
    if product_price:
        price_text = product_price.text.strip()
    else:
        price_text = 'N/A'
        
    condition_span = soup.find('div', class_='x-item-condition-value')
    if condition_span:
        condition_text = condition_span.text.strip()
    else:
        condition_text = 'N/A'
        
    product_qty = soup.find('div', class_='d-quantity__availability')
    if product_qty:
        qty_text = product_qty.text.strip()
    else:
        qty_text = 'N/A'
        
    info = soup.find_all('div', class_='ux-labels-values__values-content')
    if len(info) > 1:
        shipping_text = info[0].text.strip()
        import_charges_text = info[1].text.strip()
        delivery_text = info[2].text.strip()
        return_policy_text = info[3].text.strip()
    else:
        shipping_text = 'N/A'
        import_charges_text = 'N/A'
        delivery_text = 'N/A'
        return_policy_text = 'N/A'
        
        
    # Save extracted data in lists
    titles.append(product_title_text)
    prices.append(price_text)
    conditions.append(condition_text)
    quantities.append(qty_text)
    shippings.append(shipping_text)
    import_charges.append(import_charges_text)
    deliveries.append(delivery_text)
    return_policies.append(return_policy_text)

In [10]:
# Create a dictionary containing the data
data = {'Product Title': titles,
        'Price': prices,
        'Product Condition': conditions,
        'Quantity': quantities,
        'Shipping Info': shippings,
        'Import Charges': import_charges,
        'Delivery Info': deliveries,
        'Returns Info': return_policies}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

# Print the DataFrame
df.head()

,Product Title,Price,Product Condition,Quantity,Shipping Info,Import Charges,Delivery Info,Returns Info
0,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,5D10K37833 Lenovo Miix 700-12ISK Assembly 80QL...,EUR 149.99,Open boxOpen box“A-Grade - Vom NEUEN originale...,6 available / 9 sold,EUR 25.00 (approx US $27.27) Economy Shipping....,International shipment of items may be subject...,"Estimated between Fri, Jul 14 and Fri, Jul 28 ...",30 days returns. Buyer pays for return shippin...
2,Used - Lenovo ThinkCentre M83 - Intel Core i3 ...,US $35.00,UsedUsed“Pulled as a working system that was u...,5 available / 9 sold,US $169.47 eBay International Shipping. See de...,This item may be subject to import charges and...,"Estimated between Tue, Aug 15 and Tue, Sep 5 t...",30 days returns. Buyer pays for return shippin...
3,Lenovo IdeaCentre 3 07IAB7 Small Desktop PC i5...,GBP 445.99,NewNew,N/A,GBP 117.87 (approx US $149.57) Expedited Shipp...,Est. £89.23 Amount confirmed at checkoutThis a...,"Estimated between Fri, Jul 14 and Tue, Jul 18 ...",30 days returns. Buyer pays for return shippin...
4,"New/Orig Lenovo ThinkPad X13 Yoga Gen 1 13.3"" ...",US $179.99,NewNew,Last One / 2 sold,US $30.00 Expedited International Shipping. Se...,International shipment of items may be subject...,"Estimated between Thu, Jul 13 and Fri, Jul 21 ...",30 days returns. Buyer pays for return shippin...


In [11]:
# Save the DataFrame to a CSV file
df.to_csv('ebay_products.csv', index = False)
print('File Created Successfuly')

File Created Successfuly
